In [19]:
class Ticket:
    def __init__(self, id:int):
        self.id = id

    def __str__(self):
        return f"ticket id: {self.id}"

In [20]:
class TicketMachine:
    def __init__(self):
        self.tickets = []
        self.index = 0
    
    def __str__(self):
        return f"tickets: {[ticket.__str__() for ticket in self.tickets]}"
    
    def getNewTicket(self):
        new_ticket = Ticket(len(self.tickets))
        self.tickets.append(new_ticket)
        return new_ticket
    
    def nextTicket(self):
        self.index += 1
        return self.getCurrentTicket()
    
    def getCurrentTicket(self):
        if self.index < len(self.tickets):   
            return self.tickets[self.index]
        else:
            return None

In [21]:
gep = TicketMachine()

In [22]:
sorszam = gep.getNewTicket()
print(sorszam)
print(f"A gepben levo sorszamok: {gep}")

ticket id: 0
A gepben levo sorszamok: tickets: ['ticket id: 0']


In [23]:
lista = [1, 2,  3]

[i*i for i in lista] # Ezt amugy hivjak list comprehension-nek a pythonosok

[1, 4, 9]

In [24]:
class Customer:
    def buyTicket(self, machine:TicketMachine):
        self.ticket = machine.getNewTicket()
        
    def showTicket(self):
        return self.ticket
    

In [25]:
bela_ba = Customer()
bela_ba.buyTicket(gep)
print(bela_ba.ticket)

jozsi_ba = Customer()
jozsi_ba.buyTicket(gep)
print(jozsi_ba.ticket)

ticket id: 1
ticket id: 2


In [26]:
class Staff:
    def __init__(self, machine:TicketMachine):
        self.machine = machine
        self.busy = False
        self.currentTicket = None

    def validateTicket(self, ticket:Ticket):
        if ticket == self.currentTicket:
            return True
        return False
    
    def shoutoutTicket(self):
        if self.currentTicket is None:
            self.currentTicket = self.machine.getCurrentTicket()
            self.machine.nextTicket()
        return self.currentTicket
    
    def invalidateTicket(self):
        self.currentTicket = None
        self.busy = False
        
    def dealwithTicket(self, ticket: Ticket):
        self.busy = self.validateTicket(ticket)
        return self.busy
        

In [27]:
a = None
if a is None:
    print("a is none")
if a == None:
    print("a == none")

a is none
a == none


In [28]:
jolika = Staff(gep)
gizike = Staff(gep)


In [29]:
print(jolika.shoutoutTicket())
print(jolika.validateTicket(bela_ba.showTicket()))
print(jolika.validateTicket(jozsi_ba.showTicket()))
print(jolika.validateTicket(sorszam))

print(gizike.shoutoutTicket())
print(gizike.validateTicket(bela_ba.showTicket()))
print(gizike.validateTicket(jozsi_ba.showTicket()))
print(gizike.validateTicket(sorszam))

ticket id: 0
False
False
True
ticket id: 1
True
False
False


In [30]:
jolika.invalidateTicket()

In [31]:
def workflow_01_test():
    """
    Test01:
     Given:
       Kinyit a patika, bekapcsolják a sorszámgépet.
       Ibolya és Péter, két alkalmazott beállnak a pult mögé, és készek fogadni az ügyfeleket
       Robi bá és Erzsi, két ügyfél bejönnek a boltba

       Robi bá és Erzsi rendre húznak egy-egy sorszámot

     When:
       Ibolya és Péter, mivel épp szabadok, kérik a következőt
     Then:
        A két kiadott sorszámból egyiket Ibolya, a másikat Péter szólítja

     When:
       A két ügyfél összekeveri a pultot, ahová menniük kéne
      Then:
       A pultnál az alkalmazottak udvariasan elküldik az ügyfelet, mivel nem őt szólították

     When:
       A két ügyfél rendre oda megy a pulthoz, amelyikből szólították a sorszámukkal őket
     Then:
        Az alkalmazottak elfogadják a sorszámot, amelyet az ügyfelek bemutattak
        Az alkalmazottak elfoglaltak az ügyfelek kiszolgálásával

    """

    #shop opens and machine starts. 
    sorszamgep = TicketMachine()
    print(sorszamgep)
    #staff members are ready at the cashier
    ibolya = Staff(sorszamgep)
    peter = Staff(sorszamgep)

    #customers arrived
    robi_ba = Customer()
    erzsi = Customer()

    #customers request numbers
    robi_ba.buyTicket(sorszamgep)
    print(f"Robi bá sorszáma:{robi_ba.showTicket()}")
    erzsi.buyTicket(sorszamgep)
    print(f"Erzsi sorszáma:{erzsi.showTicket()}")
    assert erzsi.showTicket() is not robi_ba.showTicket()

    #staff members ask for new customers
    print(f"Ibolya is waiting for: {ibolya.shoutoutTicket()}")
    print(f"Peter is waiting for: {peter.shoutoutTicket()}")
    assert ibolya.shoutoutTicket() is not peter.shoutoutTicket()

    #staff members deal with customers
    print(f"Erzsi jön Ibolyához: {ibolya.dealwithTicket(erzsi.showTicket())}")
    assert ibolya.dealwithTicket(erzsi.showTicket()) is False

    print(f"Robi bá jön Peterhez: {peter.dealwithTicket(robi_ba.showTicket())}")
    assert peter.dealwithTicket(robi_ba.showTicket()) is False

    print(f"Robi bá jön Ibolyához: {ibolya.dealwithTicket(robi_ba.showTicket())}")
    assert ibolya.dealwithTicket(robi_ba.showTicket()) is True
    assert ibolya.busy is True
    
    print(f"Erzsi jön Peterhez: {peter.dealwithTicket(erzsi.showTicket())}")
    assert peter.dealwithTicket(erzsi.showTicket()) is True
    assert peter.busy is True

workflow_01_test()

tickets: []
Robi bá sorszáma:ticket id: 0
Erzsi sorszáma:ticket id: 1
Ibolya is waiting for: ticket id: 0
Peter is waiting for: ticket id: 1
Erzsi jön Ibolyához: False
Robi bá jön Peterhez: False
Robi bá jön Ibolyához: True
Erzsi jön Peterhez: True


In [32]:
def workflow_02_test():
    """
     Given:
        A sorszámgép már be van kapcsolva
        A két alkalmazott a pultnál, Ibolya és Péter épp elfoglaltak

     When:
        Jocika bejön a boltba és húz egy sorszámot, majd vár
        Egy rövid idő után megunja a várakozást és odamegy az egyik pulthoz
     Then:
        Az alkalmazott udvariasan a törelmét kéri, mivel még dolga van (talán előző ügyfél, mindegy is)

    """

    # Given:
    sorszamgep = TicketMachine()
    print(sorszamgep)
    
    ibolya = Staff(sorszamgep)
    peter = Staff(sorszamgep)

    ibolya.busy = True
    peter.busy = True

    # When:
    jocika = Customer()
    jocika.buyTicket(sorszamgep)
    # Then:
    assert ibolya.dealwithTicket(jocika.showTicket()) is False


workflow_02_test()

tickets: []


In [33]:
def workflow_03_test():
    """
     Given:
        A sorszámgép már be van kapcsolva
        Egyetlen alkalmazott van a pultnál, Ibolya
        Józsi, egy alkalmazott bejön a boltba, húz egy sorszámot és vár arra, hogy szólítsák
        Józsi jelenleg a soron következő ügyfél
     When:
        Ibolya felszabadul, kéri a következőt
     Then:
        Józsi sorszámát írja ki a gép és Ibolya Józsi sorszámát szólítja
        Józsi felmutatja a sorszámát Ibolyának, 
            amit ő elfogad 
            és elkezdi kiszolgálni

    """

    # Given:
    sorszamgep = TicketMachine()
    
    ibolya = Staff(sorszamgep)
    ibolya.busy = True

    jozsi = Customer()
    jozsi.buyTicket(sorszamgep)
    print(f"Jozsi sorszama: {jozsi.showTicket()}")
    # When:
    ibolya.busy = False

    # Then:
    assert ibolya.shoutoutTicket() is jozsi.showTicket()
    print(f"Ibolya kéri a következőt: {ibolya.shoutoutTicket()}")
    assert ibolya.dealwithTicket(jozsi.showTicket()) is True
    assert ibolya.busy is True

    return True


workflow_03_test()

Jozsi sorszama: ticket id: 0
Ibolya kéri a következőt: ticket id: 0


True

In [34]:
def workflow_04_test():
    """
     Given:
        A sorszámgép már be van kapcsolva
        Egyetlen alkalmazott van a pultnál, Ibolya
        Ibolya épp elfoglalt egy korábbi ügyféllel, Beátával
        Józsi, egy alkalmazott bejön a boltba, húz egy sorszámot és vár arra, hogy szólítsák
        Józsi jelenleg a soron következő ügyfél
     When:
        Ibolya felszabadul, kéri a következőt
     Then:
        Józsi sorszámát írja ki a gép és Ibolya Józsi sorszámát szólítja
        Józsi felmutatja a sorszámát Ibolyának, 
            amit ő elfogad 
            és elkezdi kiszolgálni

    """

    # Given:
    sorszamgep = TicketMachine()
    
    ibolya = Staff(sorszamgep)
    beata = Customer()
    beata.buyTicket(sorszamgep)

    jozsi = Customer()
    jozsi.buyTicket(sorszamgep)
    print(f"Jozsi sorszama: {jozsi.showTicket()}")
    
    ibolya.shoutoutTicket()
    ibolya.dealwithTicket(beata.showTicket())

    print(f"Ibolya épp foglalt? {ibolya.busy}")

    # When:
    print(f"A következő sorszám: {sorszamgep.getCurrentTicket()}")
    assert sorszamgep.getCurrentTicket() is jozsi.showTicket()
    print("Ibolya már kiszolgálta Beátát")
    ibolya.invalidateTicket()
    print(f"Ibolya épp foglalt? {ibolya.busy}")

    # Then:
    print(f"Ibolya kéri a következőt: {ibolya.shoutoutTicket()}")
    assert ibolya.shoutoutTicket() is jozsi.showTicket()
    assert ibolya.dealwithTicket(jozsi.showTicket()) is True
    assert ibolya.busy is True

    return True


workflow_04_test()

Jozsi sorszama: ticket id: 1
Ibolya épp foglalt? True
A következő sorszám: ticket id: 1
Ibolya már kiszolgálta Beátát
Ibolya épp foglalt? False
Ibolya kéri a következőt: ticket id: 1


True

In [35]:
def workflow_05_test():
    """
     Given:
        A sorszámgép már be van kapcsolva
        Egyetlen alkalmazott van a pultnál, Ibolya
        Ibolya épp elfoglalt egy korábbi ügyféllel, Beátával
        Józsi, egy alkalmazott bejön a boltba, húz egy sorszámot és vár arra, hogy szólítsák
        Józsi jelenleg a soron következő ügyfél
     When:
        Ibolya felszabadul, kéri a következőt
     Then:
        Józsi sorszámát írja ki a gép és Ibolya Józsi sorszámát szólítja
        Józsi felmutatja a sorszámát Ibolyának, 
            amit ő elfogad 
            és elkezdi kiszolgálni

    """

    # Given:
    sorszamgep = TicketMachine()
    
    ibolya = Staff(sorszamgep)
    beata = Customer()
    beata.buyTicket(sorszamgep)
    ibolya.shoutoutTicket()
    ibolya.dealwithTicket(beata.showTicket())

    print(f"Ibolya épp foglalt? {ibolya.busy}")

    jozsi = Customer()
    jozsi.buyTicket(sorszamgep)
    print(f"Jozsi sorszama: {jozsi.showTicket()}")
    
    # When:
    print("Ibolya már kiszolgálta Beátát")
    ibolya.invalidateTicket()
    print(f"Ibolya épp foglalt? {ibolya.busy}")

    # Then:
    print(f"A következő sorszám: {sorszamgep.getCurrentTicket()}")
    assert sorszamgep.getCurrentTicket() is jozsi.showTicket()
    print(f"Ibolya kéri a következőt: {ibolya.shoutoutTicket()}")
    assert ibolya.shoutoutTicket() is jozsi.showTicket()
    assert ibolya.dealwithTicket(jozsi.showTicket()) is True
    assert ibolya.busy is True

    return True


workflow_05_test()

Ibolya épp foglalt? True
Jozsi sorszama: ticket id: 1
Ibolya már kiszolgálta Beátát
Ibolya épp foglalt? False
A következő sorszám: ticket id: 1
Ibolya kéri a következőt: ticket id: 1


True